# Notebook 3: Value Function Iteration with Stochastic Shocks

In [ ]:
using LaTeXStrings
using Plots;

In [ ]:
# Structural parameters
β = 0.99     #discount rate
δ = 0.025    #depreciation rate
α = 0.33     #capital share
γ = 1.       #EIS
A = 1.       #mean of technology in levels
ρ = 0.9      #persistence of technology
σ = 0.008;   #SD of technological shocks

In [ ]:
# Steady-state values
kstat = ((1-β*(1-δ))/(A*β*α))^(1/(α-1))
cstat = A*kstat^α - δ*kstat;

In [ ]:
# Rouwenhorst's method to approximate AR(1) processes.
# credit@QuantEcon
function rouwenhorst(N::Integer, ρ::Real, σ::Real, μ::Real=0.0)
    σ_y = σ / sqrt(1-ρ^2)
    p  = (1+ρ)/2
    ψ = sqrt(N-1) * σ_y
    m = μ / (1 - ρ)

    state_values, p = _rouwenhorst(p, p, m, ψ, N)
    return p, [state_values;]
end

function _rouwenhorst(p::Real, q::Real, m::Real, Δ::Real, n::Integer)
    if n == 2
        return [m-Δ, m+Δ],  [p 1-p; 1-q q]
    else
        _, θ_nm1 = _rouwenhorst(p, q, m, Δ, n-1)
        θN = p    *[θ_nm1 zeros(n-1, 1); zeros(1, n)] +
             (1-p)*[zeros(n-1, 1) θ_nm1; zeros(1, n)] +
             q    *[zeros(1, n); zeros(n-1, 1) θ_nm1] +
             (1-q)*[zeros(1, n); θ_nm1 zeros(n-1, 1)]

        θN[2:end-1, :] ./= 2

        return range(m-Δ, stop=m+Δ, length=n), θN
    end
end;

In [ ]:
# Discretize the stochastic process for technology
na=7  #number of technological states
Π,a_vec=rouwenhorst(na,ρ,σ,log(A));

In [ ]:
# Discretize capital
k_low  = 0.8*kstat                 #lowest value for capital
k_high = 1.2*kstat                 #highest value for capital
nk     = 501                       #number of grid points
dk     = (k_high-k_low)/(nk - 1)   #step size
k_vec  = [k_low:dk:k_high;]        #capital vector
ks     = k_low:dk:k_high;          #range for plots

In [ ]:
# Compute returns for all point in the state space
k0 = ones(nk,1) *k_vec'          #capital today (state)
k1 = k_vec*ones(1,nk)            #capital for the next period
#consumption over all states and controls
c = cat([exp.(a_vec[i]).* k0.^α .- k1 .+ (1-δ) .* k0 for i=1:na]..., dims=3)
c[findall(x->x<0,c)] .= 0.       #set negative consumption values to zero
#period utility
if γ == 1
    u = log.(c)
else
    u = (c.^(1-γ).-1)/(1-γ)
end;

In [ ]:
# Initialize VFI algorithm
tol  = 1e-4                #tolerance for the fixed-point of V
iter = 0                   #iteration counter
dist = [10e6]              #initialize a measure of distance between iterates of V
V    = zeros(nk,na)        #initial guess for the value function
P    = zeros(nk,na);       #initialize a vector for the policy

In [ ]:
# VFI algorithm
while dist[end] > tol

    EV = V * Π' #expected continuation value
    W  = u .+ β.*repeat(reshape(EV, nk,1, na), 1, nk, 1)

    TV, TP = findmax(W,dims=1)
    #maximize for a given stock of capital and state of technology
    #(i.e., for each point in dims=2,3 of W, finds the maximum over rows.)

    TV = TV[1,:,:]
    TP = getindex.(TP[1,:,:],1)

    dist = [dist; maximum(abs.(TV - V))]

    #update policy and value functions
    V  = TV
    P  = TP

    iter+=1

end
dist=dist[2:end];

In [ ]:
println("Value Function Iteration converged in $iter iterations")

In [ ]:
# Value function
surface(exp.(a_vec), k_vec, V;
    xlabel="Z",
    ylabel="K",
    zlabel="",
    title="Value function: V(K,Z)",
    grid=false,
    camera=(55, 20),
    colorbar=false)

In [ ]:
# Policy function
K_vec = repeat(k_vec, 1, na)
K_star= K_vec[P]
surface(exp.(a_vec), k_vec, K_star;
        xlabel="Z",
        ylabel="K",
        zlabel="",
        title="Policy function: K'(K,Z)",
        grid=false,
        camera=(55, 20),
        colorbar=false)

In [ ]:
# Policy function as "slices"
plot(k_vec,K_star[:,1],label=L"z_1")
plot!(k_vec,K_star[:,7],label=L"z_7")

In [ ]:
#Euler Equation Errors
A_vec=exp.(repeat(reshape(a_vec,1, na), nk, 1))
C_0 = (A_vec.*K_vec).^α - K_star + (1-δ) .* K_vec
C_1 = C_0[P]*Π'
MPK = (α.*K_star.^(α-1).+(1-δ))*Π'
foc_err=C_0.^(-γ) - β.*C_1.^(-γ).*(MPK);

In [ ]:
# Plot the errors
plot(foc_err[:,7],label="")